In [1]:
import numpy as np

import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 115)
pd.set_option('display.width', 250)

from sklearn.ensemble import RandomForestClassifier

In [301]:
trainRaw = pd.read_csv('train_final.csv')

testData = pd.read_csv('test_final.csv')

In [592]:
#Drop Duplicate Inputs
trainData = trainRaw.drop_duplicates(subset=trainRaw.columns.difference(['income>50K']),keep=False).reset_index(drop=True)
x = len(trainData.index)

In [600]:
#Keep Duplicate Inputs
trainData = trainRaw
x = len(trainData.index)

In [569]:
#Ignore Training Data with ?
trainData = trainData.replace(to_replace = "?",value=np.nan)
trainData = trainData.dropna().reset_index(drop=True)
x = len(trainData.index)
data = trainData.append(testData.iloc[:,1:],ignore_index=True)

In [585]:
#Replace missing data with mode of column
data = trainData.append(testData.iloc[:,1:],ignore_index=True)
for col in data.columns:
    data[col] = data[col].replace(to_replace="?",value=data[col].mode()[0])

In [601]:
#Replace missing data with mode matching output
for row in range(len(trainData.index)):
    for col in range(len(trainData.columns)):
        if(trainData.iloc[row,col] == "?"):
            subTrainData = trainData[trainData['income>50K'] == trainData.iloc[row,-1]]
            mode = subTrainData[trainData.columns[col]].mode()[0]
            trainData.iloc[row,col] = mode

data = trainData.append(testData.iloc[:,1:],ignore_index=True)
for col in data.columns:
    data[col] = data[col].replace(to_replace="?",value=data[col].mode()[0])
    
#print(data)

In [602]:
#Drop fnlwgt and eduction.num, the former because random, the latter because redundant
data = data.drop(['fnlwgt','education.num'],axis=1)

#Split numerical data
data['age'] = pd.cut(x=data['age'],bins = [float('-inf'),data['age'].median(),float('inf')])
data['capital.gain'] = pd.cut(x=data['capital.gain'],bins = [float('-inf'),data['capital.gain'].median(),float('inf')])
data['capital.loss'] = pd.cut(x=data['capital.loss'],bins = [float('-inf'),data['capital.loss'].median(),float('inf')])
data['hours.per.week'] = pd.cut(x=data['hours.per.week'],bins = [float('-inf'),data['hours.per.week'].median(),float('inf')])
#print(data)

In [603]:
datDummies = pd.get_dummies(data.iloc[:,:-1])
datDummies = (datDummies-datDummies.min())/(datDummies.max()-datDummies.min())
#print(datDummies)

X = datDummies.iloc[:x,:]
y = data['income>50K'][:x]
test = datDummies.iloc[x:,:]
print(len(datDummies.index))
print(X.shape)
N = X.shape[1]
print(len(y.index))
print(len(test.index))

48842
(25000, 107)
25000
23842


In [604]:
n = 200
d = 10
#c = "entropy"
c = 'gini'


clf = RandomForestClassifier(criterion=c,max_depth=d,n_estimators=n)
clf = clf.fit(X,y)

In [605]:
yPred = clf.predict(test)
print(yPred)

[0. 0. 0. ... 1. 0. 1.]


In [606]:
valid = np.genfromtxt('validation.csv')
print(valid)

[0. 0. 0. ... 1. 0. 0.]


In [607]:
count = 0
n = len(yPred)
for i in range(n):
    if(yPred[i] == valid[i]):
        count += 1
print(f'{count/n*.91784:.5f}')

0.77005
